# Portfolio Sizing Formulation

## 1. Notations

### 1) Symbol level properties. 

For each symbol $s_i$, we have:

* $\sigma_i^{SR}$ -- Daily specific risk
* $\sigma_i^{TR}$ -- Daily total risk
* $V_i$ -- ADV value (in dollar)

### 2) Symbol level inputs.

For each symbol $s_i$, we have the following inputs:

* $C_i$ -- Capacity
* $A_i$ -- Alpha proxy
* $m_i^{\alpha}$ -- Alpha multiplier (Default is 0.3)
* $T_i^S$ -- Start datetime
* $T_i^E$ -- End datetime

### 3) Symbol level derived quantities.

For each symbol $s_i$, based on the above properties and inputs, we can calculate the following quantities:

* Duration of the Alpha signal: $T_i=T_i^E-T_i^S$
* Remaining datetime $t_i=T_i^E-TODAY$
* Alpha: $\alpha_i^T$=AlphaFunction(\*args)
    > AlphaFunction() could be customized with default: $sign(A_i)*m_i^{\alpha}*(\sigma_i^{TR}\sqrt{T_i})*\sqrt{|A_i|}$
* Expected remaining Alpha: $\alpha_i^t=AlphaDecayFunction(\alpha_i^T, T_i, t_i)$
    > AlphaDecayFunction() could be customized with default: linear decay
* Expected remaining specific risk: $\sigma_i^t=RiskDecayFunction(\sigma_i^{SR}, T_i, t_i)$
    > RiskDecayFunction() could be customized with default: square root decay
* Annualized adjustment factor: $f_i=$ Total Biz Datetime $\big/ t_i$
* Annualized Alpha: $\alpha_i^{ANN}=\alpha_i^t*f_i$
* Annualized Risk: $\sigma_i^{ANN}=\sigma_i^t*\sqrt{f_i}$

### 4) Parameters across all symbols
* $m^C$ -- Capacity multiplier
* $H$ -- Hard dollar limit
* $m^l$ -- Liquidity multiplier
* $B$ -- Risk budget
* $U^{\alpha}$ -- Alpha threshold. (Default is not used)

### 5) Portfolio level parameters
* $\theta$ -- Portfolio Sharpe requirement

### 6) Portfolio analytics

Suppose in the portfolio, there are $n$ symbols. And the notional for symbol $s_i$ is $v_i$.

* $\mathbf{v}=[|v_1|,...,|v_n|]^{T}$ -- Portfolio **GROSS** vector 
* $\mathbf{\alpha}=[|\alpha_1^{ANN}|,...,|\alpha_n^{ANN}|]^{T}$ -- Annualized **ABS** Alpha vector
* $\mathbf{\Delta}=Diag(\sigma_i^{ANN},...,\sigma_n^{ANN})$ -- Annualized specific risk matrix
* $\mathbf{\alpha^t}=[|\alpha_1^t|,...,|\alpha_n^t|]^{T}$ -- Expected remaining **ABS** Alpha vector

Note that the elements in the above vectors are all **absolute values**.

* $\alpha_P^{ANN}=\mathbf{\alpha}^T\mathbf{v}$ -- Portfolio Annualized Alpha (in dollar)
* $\alpha_P^t=\mathbf{\alpha^t}^T\mathbf{v}$ -- Portfolio Annualized Alpha (in dollar)

## 2. Symbol Level Constraints

### 1) Capacity

$\begin{aligned}
|v_i|  & \le m^C C_i\\
|v_i|  & \le H \\
\end{aligned}$

### 2) Risk

$\begin{aligned}
|v_i|  & \le \frac{B}{\sigma_i^t}\\
\end{aligned}$

### 3) Liquidity

$\begin{aligned}
|v_i|  & \le m^l V_i\\
\end{aligned}$

All the above constraints could be combined in matrix form:

$\begin{aligned}
\mathbf{I} \mathbf{v} & \le \mathbf{C}\\
\end{aligned}$

where $\mathbf{I}$ is the identity matrix, and $\mathbf{C}$ is a vector containing the tightest element-wise boundaries among the above constraints.

## 3. Portfolio Sizing Problem Setting

### 1) Portfolio Annualized Risk

Assume that the annualized common risk of the hedged portfolio could be controlled well to the level of a percentage of the portfolio gross value. Here we use $r$ to denote the percentage. Then the estimated hedged portfolio annualized total risk can be formulized as:

$\begin{aligned}
\sigma_P^{ANN} &= \sqrt{\mathbf{v}^T\mathbf{\Delta}\mathbf{\Delta}^T\mathbf{v}+(r\mathbf{1}^T\mathbf{v})^2} \\
               &= \sqrt{\mathbf{v}^T(\mathbf{\Delta}\mathbf{\Delta}^T+r^2\mathbf{J})\mathbf{v}}
\end{aligned}$

where $\mathbf{J}$ is the matrix of all ones.

### 2) Utility Function

$\begin{aligned}
\text{max } \alpha_P^t=\Sigma_i |v_i||\alpha_i^t|\\
\end{aligned}$

or

$\begin{aligned}
\text{min } -\mathbf{\alpha^t}^T \mathbf{v}\\
\end{aligned}$

### 3) Constraints

* Symbol level constraints:  $\mathbf{I} \mathbf{v} \le \mathbf{C}$
* Portfolio Sharpe requirement constraints: $\theta \sigma_P^{ANN} \le \alpha_P^{ANN}$ or $\|\theta \mathbf{L}^T \mathbf{v}\|_2 \le \mathbf{\alpha}^T\mathbf{v}$ 

where $\mathbf{L}\mathbf{L}^T=\mathbf{\Delta}\mathbf{\Delta}^T+r^2\mathbf{J}$ and $\mathbf{L}$ could be obtained by **Cholesky Decomposition**.

## 4. Implementation

The above problem setting is a typical **SOCP** optimization problem with:

* $\mathbf{c}=-\mathbf{\alpha^t}$
* $\mathbf{G}_0=\mathbf{I}, \mathbf{h}_0=\mathbf{C}$ (in **CVXOPT**, they are denoted as $\mathbf{G}_l$ and $\mathbf{h}_l$ seperately)
* $\mathbf{G}_1=[-\mathbf{\alpha}, -\theta \mathbf{L}]^T, \mathbf{h}_1=\mathbf{0}$